In [21]:
import pandas as pd
import numpy as np

In [22]:
FIFA = pd.read_csv('fifa21_train.csv')

In [23]:
FIFA.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [24]:
def cleanHightWeightFoot(df):
    #Height
    player_info_part_clean = df.copy()
    def feet2cm (x):
        height = x.split("'")
        return float(height[0])*30.48 + float(height[1][:-1])*2.54  
    player_info_part_clean['height'] = player_info_part_clean['height'].map(feet2cm)

    #Weight
    player_info_part_clean2 = player_info_part_clean.copy()
    def libs2kg (x):
        return float(x[:-3]) * 0.45359237
    player_info_part_clean2['weight'] = player_info_part_clean2['weight'].map(libs2kg)

    # Left and right foot
    player_info_part_clean3 = player_info_part_clean2.copy()
    mapping = {'Right':0,'Left':1}
    player_info_part_clean3 = player_info_part_clean3.replace({'foot': mapping})
    player_info_part_clean3.head()
    
    return player_info_part_clean3

In [25]:
def clean_money(df):
    
    data = df.copy()
    
    def typeConvert(x):
        x = x[1:]
        if 'K' in x:
            x = 1000*float(x[:-1])
        elif "M" in x:
            x = 1000000*float(x[:-1])
        else:
            float(x)
        return str(x)

    data['release clause'] = data['release clause'].map(typeConvert)
    data['release clause'] = data['release clause'].astype(float)

    data['wage'] = data['wage'].map(typeConvert)
    data['wage'] = data['wage'].astype(float)

    data['value'] = data['value'].map(typeConvert)
    data['value'] = data['value'].astype(float)
    
    return data

In [26]:
#This cleans the player_info 
def clean_player(x):
    x = x.drop(['Loan Date End'],axis=1)
    cols = []
    for i in range(len(x.columns)):
        cols.append(x.columns[i].lower())
    x.columns = cols
        
    x.position.fillna(x.bp, inplace=True)
    x.club.fillna(x['team & contract'], inplace=True)
    x = x.drop(['joined','team & contract'],axis=1)
    
    x = cleanHightWeightFoot(x)
    x = clean_money(x)
    
    return x

In [27]:
clean_player(FIFA)

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,1,525000.0,4000.0,801000.0,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,0,8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,0,9000000.0,49000.0,15300000.0,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,13,275000.0,4000.0,694000.0,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,8,725000.0,2000.0,1400000.0,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,5,625000.0,3000.0,891000.0,2018 ~ 2021,224,62,24,51,59,28.0,248,57,49.0,29,54,59,333,78,73,68.0,53,61.0,273,31,72.0,77,68,25,266,63,59.0,51.0,54.0,39,46.0,184,59,64,61.0,61,13,14,14,6,14,1589,347,3 ★,2★,Medium,Medium,1 ★,75,28,56,59,60,69,3,48+2,48+2,48+2,55+0,52+0,52+0,52+0,55+0,54+2,54+2,54+2,58+2,56+2,56+2,56+2,58+2,63+2,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,4,1600000.0,4000.0,2900000.0,2019 ~ 2022,84,12,14,13,29,16.0,96,17,15.0,19,27,18

In [28]:
#This cleans the skills part of df

def clean_skills(df):
    skills = df.copy()
    #Dropping columns
    skills = skills.drop(['attacking', 'skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis = 1)

    #Dropping NaNs
    skills = skills.dropna()
    skills = skills.reset_index(drop = True)

    #Replacing NaNs for the mean
    skills = skills.fillna(skills.mean())
    skills['composure'] = skills['composure'].round(decimals = 1)
    
    return skills

In [29]:
def cleanPlayerBaseStats(df):
    # Meaning of the columns:
    # W/F - Weak Foot
    # SM - Skill Moves
    # A/W - Attacking Work Rate
    # D/W - Defending Work Rate
    # IR -  Interception rate?
    # PAC - Pace .. player speed
    # SHO - Shoot
    # DRI - Dribling
    # DEF - Defense
    # PHY - Physical
    # Hits - 
    player_base_stats = df.copy()
   
    player_base_stats = player_base_stats.dropna()
    player_base_stats.reset_index(drop = True)

    player_base_stats_clean = player_base_stats.copy()
    removeStars1 = lambda x: x[:-1]
    removeStars2 = lambda x: x[:-2]
    player_base_stats_clean['w/f'] = player_base_stats_clean['w/f'].map(removeStars2)
    player_base_stats_clean['sm'] = player_base_stats_clean['sm'].map(removeStars1)
    player_base_stats_clean['ir'] = player_base_stats_clean['ir'].map(removeStars2)

    # We change names by data for some features (the ones that can be ordered)
    player_base_stats_clean2 = player_base_stats_clean.copy()
    mapping = {'Low':0,'Medium':1,'High':2}
    player_base_stats_clean2 = player_base_stats_clean2.replace({'a/w': mapping})
    player_base_stats_clean2 = player_base_stats_clean2.replace({'d/w': mapping})

    # We convert them to numeric values
    player_base_stats_clean3= player_base_stats_clean2.copy()
    player_base_stats_clean3['w/f'] = player_base_stats_clean3['w/f'].astype(float)
    player_base_stats_clean3['sm'] = player_base_stats_clean3['sm'].astype(float)
    player_base_stats_clean3['ir'] = player_base_stats_clean3['ir'].astype(float)

    def typeConvert(x):
        if 'K' in x:
            x = 1000*float(x[:-1])
        else:
            float(x)
        return str(x) 

    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].map(typeConvert)
    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].astype(float)

    return player_base_stats_clean3

In [30]:
def clean_positions(df):
    
    df_remaining = df.drop(['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'], axis=1)
    
    #Get the relevant data
    df_edit = df[['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova']]
    
    pos = df_edit.copy()
    #Create splits and add to the df
    pos[['ls','ls_add']] = pos.ls.str.split('+',1).tolist()
    pos[['st','st_add']] = pos.st.str.split('+',1).tolist()
    pos[['rs','rs_add']] = pos.rs.str.split('+',1).tolist()
    pos[['lw','lw_add']] = pos.lw.str.split('+',1).tolist()
    pos[['lf','lf_add']] = pos.lf.str.split('+',1).tolist()
    pos[['cf','cf_add']] = pos.cf.str.split('+',1).tolist()
    pos[['rf','rf_add']] = pos.rf.str.split('+',1).tolist()
    pos[['rw','rw_add']] = pos.rw.str.split('+',1).tolist()
    pos[['lam','lam_add']] = pos.lam.str.split('+',1).tolist()
    pos[['cam','cam_add']] = pos.cam.str.split('+',1).tolist()
    pos[['ram','ram_add']] = pos.ram.str.split('+',1).tolist()
    pos[['lm','lm_add']] = pos.lm.str.split('+',1).tolist()
    pos[['lcm','lcm_add']] = pos.lcm.str.split('+',1).tolist()
    pos[['cm','cm_add']] = pos.cm.str.split('+',1).tolist()
    pos[['rcm','rcm_add']] = pos.rcm.str.split('+',1).tolist()
    pos[['rm','rm_add']] = pos.rm.str.split('+',1).tolist()
    pos[['lwb','lwb_add']] = pos.lwb.str.split('+',1).tolist()
    pos[['ldm','ldm_add']] = pos.ldm.str.split('+',1).tolist()
    pos[['cdm','cdm_add']] = pos.cdm.str.split('+',1).tolist()
    pos[['rwb','rwb_add']] = pos.rwb.str.split('+',1).tolist()
    pos[['lb','lb_add']] = pos.lb.str.split('+',1).tolist()
    pos[['lcb','lcb_add']] = pos.lcb.str.split('+',1).tolist()
    pos[['rcb','rcb_add']] = pos.rcb.str.split('+',1).tolist()
    pos[['rb','rb_add']] = pos.rb.str.split('+',1).tolist()
    pos[['gk','gk_add']] = pos.gk.str.split('+',1).tolist()
    
    
    #Convert all columns to numeric
    for a in pos:
        pos[a] =  pd.to_numeric(pos[a], errors='coerce')
    
    #Select only columns that are relevant in respect to multicollinearity
    pos_spl_final = pos[['st',  'lw', 'cf', 'cam', 'lm','cm', 'lwb', 'cdm',  'lb', 'lcb', 'gk', 'st_add',
                             'lw_add', 'cf_add','cam_add', 'lm_add', 'cm_add', 'lwb_add', 'cdm_add', 'lb_add', 
                             'lcb_add','gk_add','ova']]
    
    df_new = pd.concat([df_remaining,pos_spl_final], axis=1)    
    
    return df_new

# Complete Function for Cleaning

In [31]:

#Global Function 
def preprocess(df):
    df_clean = df.copy()
    df_clean = clean_player(df_clean)
    df_clean = clean_skills(df_clean)
    df_clean = cleanPlayerBaseStats(df_clean)
    df_clean = clean_positions(df_clean)
    
    return df_clean

In [32]:
print(FIFA.shape)
preprocess(FIFA)

(11701, 101)


/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_3275/1956186835.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,1,525000.0,4000.0,801000.0,2015 ~ 2020,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,1682,357,4.0,2.0,2,1,1.0,69,51,63,63,51,60,3.0,58,61,62,63,63,63,59,59,58,54,15,1,0,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,0,8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,1961,412,3.0,4.0,2,0,2.0,83,75,68,82,33,71,44.0,77,77,77,76,76,68,57,53,53,48,18,0,0,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,0,9000000.0,49000.0,15300000.0,2019 ~ 2022,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,1925,404,4.0,4.0,2,1,2.0,80,77,78,86,27,56,73.0,73,80,79,80,79,74,59,56,53,41,12,2,0,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,13,275000.0,4000.0,694000.0,2016 ~ 2021,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,1527,329,2.0,2.0,1,1,1.0,57,44,54,57,57,60,7.0,50,51,51,53,53,56,56,58,57,58,14,2,0,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,8,725000.0,2000.0,1400000.0,2018 ~ 2021,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,1664,360,2.0,3.0,0,1,1.0,66,44,60,64,60,66,4.0,56,59,59,61,62,63,64,64,63,61,15,2,0,0,2,2,2,2,2,2,2,2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11417,196848,I. Ávila,29,Peru,Melgar FBC,ST,ST,170.18,69.853225,0,0,550000.0,500.0,1200000.0,2019 ~ 2021,36,72,65,57,60.0,62,56.0,56,46,63,67,65,76.0,71,89.0,64,58.0,56,60,65,50,33.0,64.0,37.0,70,60.0,41,19,20.0,12,13,8,13,14,1638,337,3.0,3.0,2,0,1.0,66,68,47,66,33,57,4.0,65,61,63,60,58,53,46,45,44,43,19,0,0,0,2,2,2,2,2,2,2,2,65
11418,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,5,625000.0,3000.0,891000.0,2018 ~ 2021,62,24,51,59,28.0,57,49.0,29,54,59,78,73,68.0,53,61.0,31,72.0,77,68,25,63,59.0,51.0,54.0,39,46.0,59,64,61.0,13,14,14,6,14,1589,347,3.0,2.0,1,1,1.0,75,28,56,59,60,69,3.0,48,55,52,54,58,56,63,60,63,61,16,2,0,0,2,2,2,2,2,2,2,2,65
11419,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,4,1600000.0,4000.0,2900000.0,2019 ~ 2022,12,14,13,29,16.0,17,15.0,19,27,18,40,40,38.0,65,54.0,50,59.0,29,64,15,30,22.0,15.0,15.0,19,56.0,12,16,13.0,72,67,66,70,72,1123,387,2.0,1.0,1,1,1.0,72,67,66,72,40,70,3.0,28,24,26,24,25,25,25,26,24,26,69,2,0,0,2,2,2,2,2,2,2,2,70
11420,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,167.64,64.863709,0,8,1100000.0,2000.0,1700000.0,2019 ~ 2024,64,66,51,64,41.0,71,57.0,38,58,67,77,76,86.0,62,87.0,61,71.0,64,37,45,27,25.0,66.0,58.0,65,58.0,37,12,13.0,13,10,11,7

In [33]:
FIFA_preprocessed = preprocess(FIFA)

/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_3275/1956186835.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


In [34]:
pd.set_option("display.max_columns", None)
FIFA_preprocessed.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,1,525000.0,4000.0,801000.0,2015 ~ 2020,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,1682,357,4.0,2.0,2,1,1.0,69,51,63,63,51,60,3.0,58,61,62,63,63,63,59,59,58,54,15,1,0,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,0,8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,1961,412,3.0,4.0,2,0,2.0,83,75,68,82,33,71,44.0,77,77,77,76,76,68,57,53,53,48,18,0,0,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,0,9000000.0,49000.0,15300000.0,2019 ~ 2022,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,1925,404,4.0,4.0,2,1,2.0,80,77,78,86,27,56,73.0,73,80,79,80,79,74,59,56,53,41,12,2,0,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,13,275000.0,4000.0,694000.0,2016 ~ 2021,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,1527,329,2.0,2.0,1,1,1.0,57,44,54,57,57,60,7.0,50,51,51,53,53,56,56,58,57,58,14,2,0,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,8,725000.0,2000.0,1400000.0,2018 ~ 2021,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,1664,360,2.0,3.0,0,1,1.0,66,44,60,64,60,66,4.0,56,59,59,61,62,63,64,64,63,61,15,2,0,0,2,2,2,2,2,2,2,2,65


# Start of Day 2

In [46]:
#First step is X - y split
#dropped some extra columns because they werent necesssary for analysiis
X = FIFA_preprocessed.drop(['id','name', 'contract', 'position','ova','nationality','club'], axis = 1)
y= FIFA_preprocessed[['ova']]

In [47]:
#First step is Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90)

In [48]:
X_train

,age,bp,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add
1980,26,CAM,177.80,74.842741,0,1,0.0,0.0,0.0,59,48,48,67,55.0,68,55.0,42,65,67,69,68,78.0,68,79.0,65,52.0,67,38,63,49,50.0,65.0,67.0,68,62.0,57,45,35.0,8,8,11,11,6,1701,354,3.0,3.0,1,1,1.0,68,57,63,69,49,48,4.0,60,65,64,66,65,64,58,57,56,50,15,1,0,0,0,1,1,1,1,1,1,1
5170,30,RM,170.18,59.874193,0,0,6500000.0,25000.0,13700000.0,77,72,59,76,64.0,77,76.0,63,74,75,67,66,75.0,74,83.0,68,57.0,67,63,69,57,56.0,75.0,77.0,62,70.0,52,56,53.0,14,15,8,16,11,1954,405,3.0,3.0,1,1,1.0,66,70,75,76,55,63,9.0,71,74,73,74,74,73,66,65,64,59,20,2,0,0,1,1,2,2,2,2,2,2
7621,24,ST,177.80,74.842741,0,6,210000.0,3000.0,378000.0,53,52,62,54,56.0,56,52.0,42,50,57,73,75,71.0,63,65.0,64,63.0,66,72,56,59,46.0,58.0,53.0,52,57.0,47,56,55.0,13,7,7,8,14,1677,360,3.0,2.0,1,1,1.0,74,56,52,59,52,67,3.0,60,59,59,57,58,55,57,55,57,57,16,2,0,0,2,2,2,2,2,2,2,2
7239,24,ST,180.34,72.121187,0,5,500000.0,2000.0,763000.0,35,67,46,58,69.0,67,56.0,42,36,66,76,68,83.0,57,79.0,57,65.0,57,58,50,36,21.0,66.0,39.0,62,57.0,29,16,19.0,15,9,12,7,7,1530,324,3.0,3.0,1,0,1.0,72,61,45,68,24,54,5.0,62,61,62,59,59,51,43,40,41,36,15,1,0,0,1,1,1,1,1,1,1,1
3411,30,CB,180.34,86.182550,0,0,3800000.0,50000.0,0.0,64,47,64,76,68.0,70,44.0,42,66,72,66,69,72.0,70,71.0,74,75.0,78,86,63,92,77.0,53.0,62.0,52,77.0,74,78,78.0,6,6,12,9,9,1945,422,3.0,2.0,1,2,2.0,68,57,66,71,75,85,4.0,64,65,65,66,67,69,73,75,73,77,15,1,0,0,1,1,1,1,1,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,32,CB,190.50,78.925072,1,0,300000.0,2000.0,525000.0,30,27,64,55,16.0,34,19.0,29,48,51,51,54,55.0,51,42.0,42,73.0,47,74,17,71,63.0,23.0,37.0,37,57.0,64,64,63.0,10,12,13,13,8,1357,296,2.0,2.0,1,2,1.0,53,28,42,42,64,67,5.0,42,40,40,42,42,46,53,57,55,64,15,2,0,0,2,2,2,2,2,2,0,2
6815,25,CAM,175.26,73.028372,0,3,8500000.0,8000.0,13000000.0,75,72,39,76,75.0,73,73.0,72,73,77,73,70,84.0,72,75.0,79,55.0,73,57,77,59,38.0,74.0,75.0,69,74.0,51,49,38.0,10,7,16,13,7,1926,402,4.0,3.0,2,1,1.0,71,75,75,75,45,61,30.0,70,74,74,75,74,72,62,61,59,51,16,2,0,0,2,2,2,2,2,2,2,2
5746,20,CAM,170.18,64.863709,0,9,120000.0,550.0,216000.0,48,42,48,63,40.0,60,48.0,39,67,61,60,69,63.0,56,83.0,52,50.0,40,40,37,44,24.0,49.0,55.0,44,57.0,37,33,43.0,6,12,5,10,15,1443,303,3.0,2.0,1,1,1.0,65,43,57,62,35,41,4.0,52,56,55,57,57,54,47,47,46,41,15,2,0,0,2,2,2,2,2,2,2,2
2717,29,GK,187.96,84.821773,0,1,25000000.0,67000.0,44400000.0,15,8,17,37,13.0,12,13.0,12,24,23,44,50,49.0,82,52.0,54,74.0,36,75,14,35,18.0,10.0,50.0,20,58.0,11,13,16.0,85,82,72,82,88,1286,456,2.0,1.0,1,1,3.0,85,82,72,88,47,82,83.0,30,28,30,32,30,32,28,30,28,29,83,3,0,0,3,3,3,3,3,3,3,2


In [49]:
numerical_train = X_train.select_dtypes(include=[np.number])
numerical_test = X_test.select_dtypes(include=[np.number])

In [50]:
categorical_train = X_train.select_dtypes(include = [object])
categorical_test = X_test.select_dtypes(include = [object])

#####      · Encoding Categorical Data.  (Not relevant)
def categoricalEncoder(df):
    from sklearn.preprocessing import OneHotEncoder
    cat = df.copy()

    # We define the function that we will use in OneHotColum
    def oneHotColumn(col_transform):
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit()
        onehotlabels = enc.transform(col_transform).toarray()
        return pd.DataFrame(onehotlabels,columns = enc.categories_)

    # We apply OneHotColum to the other features of the frame
    concat_categorical = pd.DataFrame()# categorical[['Customer']]
    for col in cat.columns:
          df_from_column = oneHotColumn(cat[[col]])
          concat_categorical = pd.concat([concat_categorical,df_from_column], axis=1)#
        
    new_columns = []
    for column in concat_categorical.columns:
        new_columns.append(column[0])
    concat_categorical.columns = new_columns
    
    return concat_categorical

In [51]:
categorical_train.columns

Index(['bp'], dtype='object')

In [ ]:
# Creating Dummy variables:


#rom sklearn.preprocessing import OneHotEncoder
#encoder_nationality = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['nationality']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_nationality.categories_)
#encoded_train_nationality = encoder_nationality.transform(categorical_train[['nationality']]).toarray()
#encoded_test_nationality = encoder_nationality.transform(categorical_test[['nationality']]).toarray()
#nationality_encoded_train = pd.DataFrame(encoded_train_nationality,columns = encoder_nationality.categories_)
#nationality_encoded_test = pd.DataFrame(encoded_test_nationality,columns = encoder_nationality.categories_)

#encoder_club = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['club']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_club.categories_)
#encoded_train_club = encoder_club.transform(categorical_train[['club']]).toarray()
#encoded_test_club = encoder_club.transform(categorical_test[['club']]).toarray()
#club_encoded_train = pd.DataFrame(encoded_train_club,columns = encoder_club.categories_)
#club_encoded_test = pd.DataFrame(encoded_test_club,columns = encoder_club.categories_)

encoder_bp = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['bp']])#fit determines how many unique alues has each categorical column. You can see below.
print(encoder_bp.categories_)
encoded_train_bp = encoder_bp.transform(categorical_train[['bp']]).toarray()
encoded_test_bp = encoder_bp.transform(categorical_test[['bp']]).toarray()
bp_encoded_train = pd.DataFrame(encoded_train_bp,columns = encoder_bp.categories_)
bp_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

categorical_encoded_train = pd.concat([nationality_encoded_train, club_encoded_train, bp_encoded_train], axis = 1)
categorical_encoded_test =

#categorical_encoded_train = pd.concat([nationality_encoded_train, club_encoded_train, bp_encoded_train], axis = 1)
#categorical_encoded_test = pd.concat([nationality_encoded_test, club_encoded_test, bp_encoded_test], axis = 1)


In [44]:
categorical_enc_train

NameError: name 'categorical_enc_train' is not defined

In [ ]:
print(categorical_train.shape)
print(categorical_test.shape)
print(categorical_encoded_train.shape)
print(categorical_encoded_test.shape)

In [ ]:
#Now is the normalization of numerical data with MinMaxScaler()
#fit train set
#transform both train and test set
from sklearn.preprocessing import MinMaxScaler 
transformer = MinMaxScaler().fit(numerical_train)
x_normalized_train = transformer.transform(numerical_train)
x_normalized_test = transformer.transform(numerical_test)
x_normalized_train = pd.DataFrame(x_normalized_train, columns=numerical_train.columns)
x_normalized_test = pd.DataFrame(x_normalized_test, columns=numerical_train.columns)


In [ ]:
x_normalized_test.head()

In [ ]:
X_transformed_train = pd.concat([x_normalized_train, categorical_encoded_train], axis = 1)
X_transformed_test = pd.concat([x_normalized_test, categorical_encoded_test], axis = 1)

In [ ]:
# 5. Modeling
#       · Apply model.
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_transformed_train, y_train)

In [ ]:
#Now performing error metrics on train data
#R2 score
from sklearn.metrics import r2_score
y_pred_train = lm.predict(X_transformed_train)

print(r2_score(y_train, y_pred_train))
#MAE ON train data
from sklearn.metrics import mean_absolute_error
mae_train = mean_absolute_error(y_train, y_pred_train)
print(mae_train)
#MSE on train DATA
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, y_pred_train)
print(mse)
#RMSE ON train DATA
import math
rmse = math.sqrt(mse)
print(rmse)

# Formular for Transformation

# Validation

In [ ]:
fifa_valid = pd.read_csv('fifa21_validate.csv')

In [ ]:
fifa_valid.head()

In [ ]:
fifa_valid_new = preprocess(fifa_valid)

In [ ]:
fifa_valid_new.head()

In [ ]:
#First step is X - y split
#dropped some extra columns because they werent necesssary for analysiis
X_2 = fifa_valid_new.drop(['id','name', 'contract', 'position','ova'], axis = 1)
y_2 = fifa_valid_new[['ova']]

In [ ]:
#First step is Train-test split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90)

In [ ]:
numerical_new = X_2.select_dtypes(include=[np.number])
categorical_new =X_2.select_dtypes(include = object)

In [ ]:
display(numerical_new.head())
display(categorical_new.head())

In [ ]:
#from sklearn.preprocessing import MinMaxScaler 
#transformer = MinMaxScaler().fit()
x_normalized_new = transformer.transform(numerical_new)

In [ ]:
#One hot encoding
#from sklearn.preprocessing import OneHotEncoder
#encoder_nationality = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['nationality']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_nationality.categories_)
encoded_nationality_new = encoder_nationality.transform(categorical_new[['nationality']]).toarray()
#encoded_test_nationality = encoder_nationality.transform(categorical_test[['nationality']]).toarray()
nationality_encoded_new = pd.DataFrame(encoded_nationality_new,columns = encoder_nationality.categories_)
#nationality_encoded_test = pd.DataFrame(encoded_test_nationality,columns = encoder_nationality.categories_)

#encoder_club = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['club']])#fit determines how many unique alues has each categorical column. You can see below.
encoded_club_new = encoder_club.transform(categorical_new[['club']]).toarray()
#encoded_test_club = encoder_club.transform(categorical_test[['club']]).toarray()
club_encoded_new = pd.DataFrame(encoded_club_new,columns = encoder_club.categories_)
#club_encoded_test = pd.DataFrame(encoded_test_club,columns = encoder_club.categories_)

#encoder_bp = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['bp']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_bp.categories_)
encoded_bp_new = encoder_bp.transform(categorical_new[['bp']]).toarray()
#encoded_test_bp = encoder_bp.transform(categorical_test[['bp']]).toarray()
bp_encoded_new = pd.DataFrame(encoded_bp_new,columns = encoder_bp.categories_)
#bp_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

categorical_encoded_new = pd.concat([nationality_encoded_new, club_encoded_new, bp_encoded_new], axis = 1)
categorical_encoded_new.head()
#categorical_encoded_test = pd.concat([nationality_encoded_test, club_encoded_test, bp_encoded_test], axis = 1)

In [ ]:
#Now is the normalization of numerical data with MinMaxScaler()
#fit train set
#transform both train and test set
#from sklearn.preprocessing import MinMaxScaler 
#transformer = MinMaxScaler().fit(numerical_train)
x_normalized_new = transformer.transform(numerical_new)
#x_normalized_test = transformer.transform(numerical_test)
x_normalized_new = pd.DataFrame(x_normalized_new, columns=numerical_train.columns)
#x_normalized_test = pd.DataFrame(x_normalized_test, columns=numerical_train.columns)

In [ ]:
x_normalized_new.head()

In [ ]:
#Concatenat

x_transformed_new = pd.concat([x_normalized_new, categorical_encoded_new], axis =1)
x_transformed_new.head()

In [ ]:
reg_predicted = reg.predict(x_transformed_new)

In [ ]:
#Now performing error metrics on train data
#R2 score
#from sklearn.metrics import r2_score
reg_predicted = reg.predict(x_transformed_new)
#y_pred_train = reg.predict(X_transformed_train)

print(r2_score(y_2, reg_predicted))
#MAE ON train data
from sklearn.metrics import mean_absolute_error
mae_new = mean_absolute_error(y_2, reg_predicted)
print(mae_new)
#MSE on train DATA
from sklearn.metrics import mean_squared_error
mse_new = mean_squared_error(y_2, reg_predicted)
print(mse_new)
#RMSE ON train DATA
import math
rmse_new = math.sqrt(mse_new)
print(rmse_new)

In [ ]:
categorical_enc_test